In [215]:
import pandas as pd 
import  numpy as np
import matplotlib as plt
import seaborn as sns
import sqlite3
from scipy import sparse
import sqlite3
from sklearn.metrics.pairwise import cosine_similarity

In [216]:
# Connect to SQLite database
conn = sqlite3.connect('../database/data.db')

In [217]:
def load_corr_matrix_from_db():
    conn = sqlite3.connect('../database/matrix.db')
    corr_matrix = pd.read_sql('SELECT * FROM correlation_matrix', conn, index_col='title')
    conn.close()
    return corr_matrix

# Load the correlation matrix from the database
corr_matrix = load_corr_matrix_from_db()

DatabaseError: Execution failed on sql 'SELECT * FROM correlation_matrix': no such table: correlation_matrix

In [218]:
# Load data from SQLite tables
ratings_query = "SELECT movieId, userId, rating FROM ratings"
movies_query = "SELECT movieId, title, year,  genres, tmdbId FROM movies"

In [219]:
ratings_query

'SELECT movieId, userId, rating FROM ratings'

In [220]:
ratings = pd.read_sql(ratings_query, conn)
movies = pd.read_sql(movies_query, conn)

In [221]:
movies.head()

,movieId,title,year,genres,tmdbId
0,1,Toy Story,1995,"Adventure, Animation, Children, Comedy, Fantasy",862
1,2,Jumanji,1995,"Adventure, Children, Fantasy",8844
2,3,Grumpier Old Men,1995,"Comedy, Romance",15602
3,4,Waiting to Exhale,1995,"Comedy, Drama, Romance",31357
4,5,Father of the Bride Part II,1995,Comedy,11862


In [222]:
movies.isna().sum()

movieId    0
title      0
year       0
genres     0
tmdbId     0
dtype: int64

In [223]:
movies['title'].duplicated().sum()

0

In [224]:
movies['tmdbId'].duplicated().sum()

0

In [225]:
movies[movies['year'] == 'Ben-hur']

,movieId,title,year,genres,tmdbId


In [226]:
movies['year'].unique()

array(['1995', '1994', '1996', '1976', '1992', '1988', '1967', '1993',
       '1964', '1977', '1965', '1982', '1990', '1991', '1989', '1937',
       '1940', '1969', '1981', '1973', '1970', '1960', '1955', '1959',
       '1968', '1980', '1975', '1986', '1948', '1943', '1950', '1987',
       '1997', '1974', '1956', '1958', '1949', '1972', '1998', '1933',
       '1952', '1951', '1957', '1961', '1954', '1934', '1944', '1963',
       '1942', '1941', '1953', '1939', '1947', '1946', '1945', '1938',
       '1935', '1936', '1932', '1985', '1979', '1971', '1978', '1966',
       '1962', '1983', '1984', '1931', '1922', '1999', '1927', '1929',
       '1930', '1928', '1925', '2000', '1919', '1926', '1923', '1920',
       '1918', '1921', '2001', '1924', '2002', '2003', '1915', '2004',
       '1916', '1917', '2005', '2006', '1902', '2007', '2008', '2009',
       '1914', '1912', '2010', '1913', '2011', '1898', '1899', '1894',
       '2012', '2013', '1896', '1910', '2014', '2015', '1895', '1909',
      

In [227]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57897 entries, 0 to 57896
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  57897 non-null  int64 
 1   title    57897 non-null  object
 2   year     57897 non-null  object
 3   genres   57897 non-null  object
 4   tmdbId   57897 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ MB


In [228]:
#movies = movies.drop_duplicates(subset=['title'])
#movies = movies.drop_duplicates(subset=['tmdbId'])

In [229]:
movies['title'].duplicated().sum()

0

In [230]:
movies['tmdbId'].duplicated().sum()

0

In [231]:
movies['movieId'].duplicated().sum()

0

In [232]:
movies

,movieId,title,year,genres,tmdbId
0,1,Toy Story,1995,"Adventure, Animation, Children, Comedy, Fantasy",862
1,2,Jumanji,1995,"Adventure, Children, Fantasy",8844
2,3,Grumpier Old Men,1995,"Comedy, Romance",15602
3,4,Waiting to Exhale,1995,"Comedy, Drama, Romance",31357
4,5,Father of the Bride Part II,1995,Comedy,11862
...,...,...,...,...,...
57892,209155,Santosh Subramaniam,2008,"Action, Comedy, Romance",63813
57893,209159,Window of the Soul,2001,Documentary,63407
57894,209163,Bad Poems,2018,"Comedy, Drama",553036
57895,209169,A Girl Thing,2001,(no genres listed),162892


In [233]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57897 entries, 0 to 57896
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  57897 non-null  int64 
 1   title    57897 non-null  object
 2   year     57897 non-null  object
 3   genres   57897 non-null  object
 4   tmdbId   57897 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ MB


In [234]:
def get_genres(movie_title):
    movie_row = movies[movies['title'] == movie_title]
    return movie_row['genres'].iloc[0]

movie_title = 'Toy Story'
genres = get_genres(movie_title)
genres

'Adventure, Animation, Children, Comedy, Fantasy'

In [235]:
def get_year(movie_title):
    movie_row = movies[movies['title'] == movie_title]
    return movie_row['year'].iloc[0]

movie_title = 'Toy Story'
genres = get_year(movie_title)
genres

'1995'

In [236]:
movies.head()

,movieId,title,year,genres,tmdbId
0,1,Toy Story,1995,"Adventure, Animation, Children, Comedy, Fantasy",862
1,2,Jumanji,1995,"Adventure, Children, Fantasy",8844
2,3,Grumpier Old Men,1995,"Comedy, Romance",15602
3,4,Waiting to Exhale,1995,"Comedy, Drama, Romance",31357
4,5,Father of the Bride Part II,1995,Comedy,11862


In [237]:
# Merge ratings with movie names
ratings_with_name = pd.merge(ratings, movies, on='movieId')
ratings_with_name.head()

,movieId,userId,rating,title,year,genres,tmdbId
0,31,1,2.5,Dangerous Minds,1995,Drama,9909
1,31,7,3.0,Dangerous Minds,1995,Drama,9909
2,31,31,4.0,Dangerous Minds,1995,Drama,9909
3,31,32,4.0,Dangerous Minds,1995,Drama,9909
4,31,36,3.0,Dangerous Minds,1995,Drama,9909


In [238]:
# Compute movie statistics
movie_stats = ratings_with_name.groupby('title').agg(num_ratings=('rating', 'count'), avg_rating=('rating', 'mean')).reset_index()
movie_stats.head()

,title,num_ratings,avg_rating
0,"""Great Performances"" Cats",2,1.750000
1,$9.99,3,3.833333
2,'Hellboy': The Seeds of Creation,1,2.000000
3,'Neath the Arizona Skies,1,0.500000
4,'Round Midnight,2,2.250000


In [239]:
M = 5
popular_movies = movie_stats[movie_stats['num_ratings'] >= M]
popular_movies.head()

,title,num_ratings,avg_rating
7,"'burbs, The",19,3.052632
9,(500) Days of Summer,45,3.755556
10,*batteries not included,7,3.142857
12,...And Justice for All,13,3.692308
14,10,5,2.400000


In [240]:
popular_movies.shape

(3384, 3)

## Populaity Based Recommondations system

In [241]:
# Calculate weighted average rating
C = popular_movies['avg_rating'].mean()  # Mean rating across all movies
popular_movies['weight_avg_rating'] = ((popular_movies['num_ratings'] / (popular_movies['num_ratings'] + M)) * popular_movies['avg_rating']) + ((M / (popular_movies['num_ratings'] + M)) * C)
popular_movies.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_13804\4162136284.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popular_movies['weight_avg_rating'] = ((popular_movies['num_ratings'] / (popular_movies['num_ratings'] + M)) * popular_movies['avg_rating']) + ((M / (popular_movies['num_ratings'] + M)) * C)


,title,num_ratings,avg_rating,weight_avg_rating
7,"'burbs, The",19,3.052632,3.128063
9,(500) Days of Summer,45,3.755556,3.721470
10,*batteries not included,7,3.142857,3.256125
12,...And Justice for All,13,3.692308,3.615195
14,10,5,2.400000,2.907351


In [242]:
popular_movies = popular_movies.sort_values(by='weight_avg_rating', ascending=False)
popular_movies.head(100)

,title,num_ratings,avg_rating,weight_avg_rating
6641,"Shawshank Redemption, The",311,4.487138,4.470169
3032,"Godfather, The",200,4.487500,4.461334
3033,"Godfather: Part II, The",135,4.385185,4.350525
7991,"Usual Suspects, The",201,4.370647,4.347444
223,"African Queen, The",50,4.420000,4.328609
...,...,...,...,...
3108,"Graduate, The",89,4.123596,4.085888
4461,"Lock, Stock & Two Smoking Barrels",74,4.128378,4.083209
3432,His Girl Friday,26,4.211538,4.083016
3677,"Ideal Husband, An",20,4.250000,4.082940


In [243]:
# Create pivot table for user-movie ratings
movies_users = pd.pivot_table(ratings_with_name, index='userId', columns='title', values='rating', fill_value=0)
movies_users

title,"""Great Performances"" Cats",$9.99,'Hellboy': The Seeds of Creation,'Neath the Arizona Skies,'Round Midnight,'Salem's Lot,'Til There Was You,"'burbs, The",'night Mother,(500) Days of Summer,...,"Zorro, the Gay Blade",Zulu,[REC],eXistenZ,loudQUIETloud: A Film About the Pixies,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us),İtirazım Var
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
2,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
3,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
4,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
5,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
668,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
669,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0


In [109]:
corr_matrix = movies_users.corr(method='pearson')
corr_matrix.head(100)

title,"""Great Performances"" Cats",$9.99,'Hellboy': The Seeds of Creation,'Neath the Arizona Skies,'Round Midnight,'Salem's Lot,'Til There Was You,"'burbs, The",'night Mother,(500) Days of Summer,...,"Zorro, the Gay Blade",Zulu,[REC],eXistenZ,loudQUIETloud: A Film About the Pixies,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us),İtirazım Var
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats",1.000000,-0.002894,-0.001718,0.162966,0.018512,-0.001718,0.011006,-0.007311,-0.002980,-0.008271,...,-0.002428,-0.002792,-0.002929,-0.008014,-0.001718,-0.007546,-0.001718,-0.009406,-0.001718,-0.001718
$9.99,-0.002894,1.000000,-0.002514,-0.002514,-0.002807,-0.002514,-0.004518,0.070014,-0.004360,0.144961,...,-0.003553,-0.004086,-0.004286,-0.011726,-0.002514,0.003087,-0.002514,0.045869,-0.002514,-0.002514
'Hellboy': The Seeds of Creation,-0.001718,-0.002514,1.000000,-0.001493,-0.001666,1.000000,-0.002682,0.214088,-0.002589,-0.010031,...,-0.002109,-0.002426,-0.002545,-0.006962,-0.001493,-0.006556,-0.001493,-0.008172,-0.001493,-0.001493
'Neath the Arizona Skies,0.162966,-0.002514,-0.001493,1.000000,0.122576,-0.001493,0.083023,-0.006351,-0.002589,0.009882,...,-0.002109,-0.002426,-0.002545,-0.006962,-0.001493,-0.006556,-0.001493,-0.008172,-0.001493,-0.001493
'Round Midnight,0.018512,-0.002807,-0.001666,0.122576,1.000000,-0.001666,0.007638,0.138760,-0.002891,0.129609,...,-0.002355,-0.002708,-0.002841,0.115866,-0.001666,-0.007320,-0.001666,-0.009124,-0.001666,-0.001666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300: Rise of an Empire,-0.003712,-0.005431,-0.003225,-0.003225,-0.003600,-0.003225,-0.005795,-0.013722,-0.005593,0.269227,...,-0.004557,0.074079,0.301910,0.042632,-0.003225,0.096988,0.408901,0.099634,-0.003225,-0.003225
35 Up,-0.003433,0.178162,-0.002983,-0.002983,0.438046,-0.002983,-0.005360,0.162693,-0.005174,0.189966,...,-0.004215,-0.004847,-0.005085,0.041409,-0.002983,0.006177,-0.002983,0.065035,-0.002983,-0.002983
"36th Chamber of Shaolin, The (Shao Lin san shi liu fang) (Master Killer)",-0.002431,-0.003558,-0.002112,-0.002112,-0.002358,-0.002112,-0.003796,-0.008989,-0.003664,0.112622,...,-0.002985,-0.003433,-0.003601,0.056264,-0.002112,0.099943,-0.002112,0.122894,-0.002112,-0.002112


In [244]:
corr_matrix.duplicated().sum()

79

In [110]:
def get_similar(movie_name,rating):
    similar_score = corr_matrix[movie_name]*(rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_score

In [112]:
def get_similar_movies(movie_names, user_ratings):
    similar_movies_list = []
    for movie_name, rating in zip(movie_names, user_ratings):
        similar_movies = get_similar(movie_name, rating).head(10)
        similar_movies_list.append(similar_movies)
    similar_movies_df = pd.concat(similar_movies_list)
    return similar_movies_df

In [114]:
num_movies = int(input("Enter the number of movies you want to rate: "))

movie_list = []
rating_list = []

for i in range(num_movies):
    movie_name = input("Enter the name of movie " + str(i+1) + ": ")
    user_rating = float(input("Enter your rating (0-10) for " + movie_name + ": "))
    movie_list.append(movie_name)
    rating_list.append(user_rating)

movie_names = np.array(movie_list)
user_ratings = np.array(rating_list)

In [119]:
similar_movies = get_similar_movies(movie_names, user_ratings)
print(similar_movies)

title
Sabrina                         0.500000
Phenomenon                      0.211340
Truth About Cats & Dogs, The    0.206100
Roman Holiday                   0.190385
Eraser                          0.188666
Tin Cup                         0.186070
Multiplicity                    0.181704
Grumpier Old Men                0.179009
Executive Decision              0.176287
American President, The         0.173159
Grumpier Old Men                0.500000
Phenomenon                      0.229561
Multiplicity                    0.219462
Executive Decision              0.202062
Nutty Professor, The            0.183702
Juror, The                      0.183680
Down Periscope                  0.180702
Sabrina                         0.179009
River Wild, The                 0.175149
Eraser                          0.171652
Father of the Bride Part II     1.500000
Truth About Cats & Dogs, The    0.607022
Executive Decision              0.574701
Down Periscope                  0.541913
River Wild